## Getting started with LoRaWAN on IoT-LAB using RIOT

The goal of this notebook is to discover the basics of LoRaWAN communication using RIOT and ChirpStack (CampusIoT) As LoRaWAN network provider.
This Notebook is divided in several steps:
1. The first step requires you to configure a LoRaWAN application with one device on the ChirpStack backend
1. You will then build and flash a RIOT application that provides a shell to control the LoRaWAN stack. On the board, you will configure the identifiers and keys required for Over-The-Air Activation (OTAA).
1. In the last step of this tutorial, you will exchange messages between the board and ChirpStack.

### LoRaWAN overview

A LoRaWAN network includes a set of subsystems and protocols allowing _end-devices_ to communicate on long distances while consuming little energy.

To communicate by radio, LoRaWAN devices and gateways use the LoRa radio technology. This technology is based on a Chirp Spreading Spectrum frequency modulation within public radio frequency bands (ISM). The access to the physical layer is regulated and depends on the region of use.

For example, LoRaWAN defines the notion of "datarate". A datarate index is the combination of a spreading factor (SF) and a bandwidth (BW), as shown in the table below:

<figure style="text-align:center">
    <img src="images/lorawan-datarate-table.png" width="650" align="center"/><br/><br/>
    <figcaption><em>LoRaWAN Datarate table for EU868 band</em></figcaption>
</figure>

A datarate of 0 uses a spreading factor of 12 and a bandwidth of 125kHz, in this case the transmission is the slowest (250 bit/s). A datarate of 6 uses a spreading factor of 7 and a bandwidth of 250kHz, the transmission is the fastest in this case (11kbit/s).

You can find more information on LoRaWAN [here](https://fr.wikipedia.org/wiki/LoRaWAN) and [here](https://docs.wixstatic.com/ugd/eccc1a_20fe760334f84a9788c5b11820281bd0.pdf):

On IoT-LAB, the EU868 ISM band is used by the boards, because they are all located in Europe and the boards can only transmit on this band.

<figure style="text-align:center">
    <img src="images/lorawan-classes.jpg" width="640" align="center"/><br/><br/>
    <figcaption><em>The LoRaWAN stack</em></figcaption>
</figure>

The LoRaWAN specifications define 3 classes of end-devices: A, B, C. Each of which has specific access to the physical layer. For more details, please refer to the LoRa specifications provided by the LoRa Alliance [here](https://www.lora-alliance.org/lorawan-for-developers).

Here is a global overview of a LoRaWAN network:

<figure style="text-align:center">
    <img src="images/lorawan-system.png" width="640" align="center"/><br/><br/>
    <figcaption><em>The LoRaWAN infrastructure</em></figcaption>
</figure>

End devices use LoRa modulation to communicate with the gateways, which themselves use the regular Internet protocols to communicate with the LoRaWAN server, owned by a provider.

### ChirpStack
ChirpStack is an open-source LoRaWAN network server stack. Differently than ChirpStack
its not a publicly built network. Gateway communicate with a privately hosted
network server that de-duplicates data and the forwards it to the Application
server. See [ChirpStack architecture](https://www.chirpstack.io/project/architecture/)
for more details.

### Configure a ChirpStack application

Before using the LoRa devices, you have to create an account and register an
application with a device configured on ChirpStack.

1. If you don’t already have a ChirpStack account, ask your server provider for
one.

2. Connect to the [Campus IoT Server](https://lns.campusiot.imag.fr/#/logink) and
register your endpoint with the name `LORA_E5_DEV_XXXX_YYYY` in the application
[**IESEG_ALARM_GZ**](https://lns.campusiot.imag.fr/#/organizations/22/applications),
`Z` matching your group number, `XXXX` are the 4 last characters of the board `DevEUI`
and `YYYY` are the the initials of each group member. The device profile can be changed
later, but select `OTAA_CLASS_A_NONE` for starters.

<figure style="text-align:center">
    <img src="images/chirpstack-device-config.png" width="640" align="center"/><br/><br/>
    <figcaption><em>Device Creation</em></figcaption>
</figure>

Once the devices is created the `AppKey` should be generated.

<figure style="text-align:center">
    <img src="images/chirpstack-appkey.png" width="640" align="center"/><br/><br/>
    <figcaption><em>Device Application Key</em></figcaption>
</figure>


> **Important:** In the following instructions of this tutorial, we consider that your application name is **_iotlab-lorawan_** and your node id is **_iotlab-node_**. 
>
> **Replace these names with the information you configured in your own ChirpStack account and application**.

### Build and flash the RIOT firmware

To use the LoRa-E5 board with the ChirpStack server, you must first flash the RIOT LoRaWAN test application, `tests/pkg_semtech-loramac`:

1. Build and flash the `tests/pkg_semtech-loramac` application of RIOT.

This application provides a shell to finely control the loramac stack on the device. By default, the target of the application corresponds to the LoRa-E5-DEV

In [ ]:
!make -C ../../RIOT/tests/pkg_semtech-loramac flash

To communicate in LoRaWAN, this application relies on a library (a "package") developed by Semtech and [available on GitHub](https://github.com/Lora-net/LoRaMac-node). As of today, this library is the reference implementation of the MAC layer of the LoRaWAN protocol for end-devices.

2. Open a terminal: `File > New > Terminal` and connect to the shell by running the following command in the terminal:

You can press *Enter* to get the `>` prompt.

You are now ready for configuring your LoRaWAN device!

### Configure the LoRaWAN device

Before going through the next actions, got to your application details on
[ChirpStack](https://lns.campusiot.imag.fr/#/organizations/22/applications) take
note of the DevEui and of the Application Key.

The RIOT shell provides the `loramac` command to control the loramac stack, join the network, and send packets:

1. In the RIOT shell, let's start by configuring the loramac stack with your DEVEUI, APPKEY, the APPEUI can be set to all 0 (copy the APPKEY as a hexstring)

 >**note:** Use the values of your registered device from the ChirpStack backend.
 >**note:** Dont forget to connect the LoRa-E5 Dev antenna!!

2. Configure a fast datarate, e.g. 5, corresponding to a bandwidth of 125kHz and a spreading factor of 7, since the nodes are very close to the gateway:

3. Now that the device is correctly configured for OTAA activation, it is time to join it to the network:

> **Reminder:** The principle of OTAA activation is to use the identifiers of the device (Device EUI) with the application key to build a secure connection request.
> Upon receipt of this connection request, the LoRaWAN server checks that the message is valid, using the Message Integrity Code (MIC), and it verifies the validity of the identifiers.
> If the request is valid, the server responds with an acknowledgement message containing a "nonce" (an arbitrary number) that will be used by the server and by the device to determine the session keys (network and application) from the initial application key.

On the ChirpStack web console, go to the **ACTIVATION** tab of the `iotlab-node` page.
You should now see that the device is now configured.

<figure style="text-align:center">
    <img src="images/chirpstack-activation.png" width="1134" align="center"/><br/><br/>
    <figcaption><em>Joined Device</em></figcaption>
</figure>

4. Now use the `tx` subcommand with the `uncnf` option to send an unconfirmed payload (no ACK expected from the server) to the backend: 

Still in the **DEVICE DATA** tab of the `iotlab-node` web page, you should see the message received by the ChirpStack backend:

<figure style="text-align:center">
    <img src="images/chirpstack-uplink.png" width="1134" align="center"/><br/><br/>
    <figcaption><em>Data Uplink</em></figcaption>
</figure>

Payload are encoded as base64 strings. You can decode it using the following snippet
(replace the value assigned to `bas64_str` with the payload received) to verify that
the message is correct:

In [ ]:
import base64
base64_str = "SGVsbG9Xb3JsZCE="
print(base64.b64decode(base64_str))

5. Send a downlink message from the **DETAILS** tab of the `iotlab-node` device (for example use the following base64 encoDED STRING: **SGVsbG9SSU9UIQ==**)

> **Note**: If the downlink is forbidden with a message saying that the device is not activated, just refresh the page.

> **Note**: Nothing is received by the node because a LoRaWAN class A device (the default for a RIOT LoRaWAN node) only have a short RX window after a send.

<figure style="text-align:center">
    <img src="images/chirpstack-downlink.png" width="1134" align="center"/><br/><br/>
    <figcaption><em>Downlink</em></figcaption>
</figure>

In order to receive the downlink message, just send another message to ChirpStack: 

You sent and received LoRaWAN messages using RIOT on LoRa-E5-DEV, congratulations !